In [1]:
import os
import sys
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import open3d.visualization as vis

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
keypoints_path = "./KITTI/keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise"
sequence = "00"
bin_num = "000000"

pcs = []
folder = os.path.join(keypoints_path, sequence)
folder = os.listdir(folder)   
folder.sort(key=lambda x:int(x[:-4]))
for idx in range(len(folder)):
    file = os.path.join(keypoints_path, sequence, folder[idx])
    # print("Loading", file)
    if os.path.isfile(file):
        pc = np.fromfile(file, dtype=np.float32).reshape((-1, 37))
        pcs.append(pc)
        # print("Loaded", pc.shape, "keypoints")
    else:
        pcs.append([0])
pcs = np.array(pcs)
print("Loaded", pcs.shape, "point clouds")

Loaded (4541, 256, 37) point clouds


In [3]:
# for pc in pcs:
kp_coords = np.array(pcs[:, :,:3])
kp_scores = pcs[:,:,3]
kp_descriptors = pcs[:,:,4:]
    # print(kp_coords.shape, kp_scores.shape, kp_descriptors.shape)
# Calculate the mean, minimum, and maximum values for kp_coords
kp_coords_mean = np.mean(kp_coords)
kp_coords_min = np.min(kp_coords)
kp_coords_max = np.max(kp_coords)

# Calculate the mean, minimum, and maximum values for scores
scores_mean = np.mean(kp_scores)
scores_min = np.min(kp_scores)
scores_max = np.max(kp_scores)

# Calculate the mean, minimum, and maximum values for descriptors
descriptors_mean = np.mean(kp_descriptors)
descriptors_min = np.min(kp_descriptors)
descriptors_max = np.max(kp_descriptors)

# Print the results
print("kp_coords - Mean:", kp_coords_mean, "Min:", kp_coords_min, "Max:", kp_coords_max)
print("scores - Mean:", scores_mean, "Min:", scores_min, "Max:", scores_max)
print("descriptors - Mean:", descriptors_mean, "Min:", descriptors_min, "Max:", descriptors_max)


kp_coords - Mean: 0.5961099 Min: -79.930115 Max: 79.729034
scores - Mean: 0.27414268 Min: 0.03911762 Max: 0.8457249
descriptors - Mean: 9.090902 Min: 0.0 Max: 100.00006


In [4]:
pc_file = os.path.join('/media/vision/Seagate/DataSets/kitti/dataset/sequences', sequence, "velodyne" ,'000000.bin')
pc = np.fromfile(pc_file, dtype=np.float32)
pc = pc.reshape((-1, 4))
pc_coords = pc[:, :3]
pc_colors = pc[:, 3]
# pc = pc[np.random.choice(pc.shape[0], 16384, replace=False), :]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc[:, :3])

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
pc_descriptors = np.transpose(pcd_fpfh.data)#[-256:]

In [5]:
# seq 0, 00 pose에서 전체 pc에 대한 descriptor와 주어진 kp_descriptors의 거리를 계산
l2_norms = np.linalg.norm(pc_descriptors - kp_descriptors[0][100], axis=1)
count = np.count_nonzero(l2_norms < 158.72552491687108)
print(count)

print("target keypoint's coord: ", kp_coords[0][100])
print("min distance: ", np.min(l2_norms))
top_5_indices = np.argsort(l2_norms)[:5]
top_5_candidates = pc[top_5_indices,:3]
print("top 5 candidates coord: ", top_5_candidates)


51291
target keypoint's coord:  [31.036074   13.310688    0.18449429]
min distance:  62.54184065245435
top 5 candidates coord:  [[ 0.86937934 -9.577588    0.07708635]
 [ 0.8387991  -9.567581    0.07708955]
 [-6.1696143  -8.652582    0.46096808]
 [ 0.81022465 -9.5815735   0.07709592]
 [ 0.79443926 -9.57457     0.07709728]]


In [6]:
import open3d as o3d
import numpy as np
# 키포인트에서 가장 가까운 점의 디스크립터와 비교

# Compute distances between target_point and pcd_points
distances = np.linalg.norm(pc_coords - kp_coords[0][100], axis=1)

# Find the index of the closest point
closest_index = np.argmin(distances)

print("Closest point's coordinates:", pc_coords[closest_index])
print("Closest point's descriptors:", pc_descriptors[closest_index][:5])
print("target keypoint's coord: ", kp_coords[0][100])
print("target keypoint's descriptors: ", kp_descriptors[0][100][:5])
print("Distance between the two descriptors:", np.linalg.norm(pc_descriptors[closest_index] - kp_descriptors[0][100]))



Closest point's coordinates: [31.148245   13.401265    0.15085755]
Closest point's descriptors: [0. 0. 0. 0. 0.]
target keypoint's coord:  [31.036074   13.310688    0.18449429]
target keypoint's descriptors:  [0.         0.         0.         0.44523013 4.9243283 ]
Distance between the two descriptors: 255.06231592112175
